# Classification using SVM
BY EDISON LIM JUN HAO

Classification of the features extracted from part (ii). 
As mentioned on the note, I could not extract the test features and perform analysis. I wish to spend more time on CNN and did not work much on this part. 

The results below are from the 80/20 train test split on the train dataset. 

In [15]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

DATA = '/Users/edison/Desktop/Yelp/project/'

train_photo_to_biz = pd.read_csv(DATA+'train_photo_to_biz_ids.csv', index_col='photo_id')


In [2]:
train_photo_to_biz.head()

,business_id
photo_id,
204149,3034
52779,2805
278973,485
195284,485
19992,485


In [3]:
train_df = pd.read_csv(DATA+"biz_label_features(train).csv")
train_df.head()

,business,label,features
0,1000.0,"(1, 2, 3, 4, 5, 6, 7)","[0.13958423, 0.085037857, 0.09016072, 0.125575..."
1,1001.0,"(0, 1, 6, 8)","[0.12032899, 0.050164513, 0.077049561, 0.11587..."
2,100.0,"(1, 2, 4, 5, 6, 7)","[0.14395453, 0.077671967, 0.087839253, 0.13833..."
3,1006.0,"(1, 2, 4, 5, 6)","[0.1636648, 0.078282505, 0.12112327, 0.0852652..."
4,1010.0,"(0, 6, 8)","[0.17404604, 0.070603535, 0.11224925, 0.115906..."


In [4]:
# Convert labels into array 
y_train = train_df['label'].values
X_train = train_df['features'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x)>0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

y_train = np.array([convert_label_to_array(y) for y in train_df['label']])
X_train = np.array([convert_feature_to_vector(x) for x in train_df['features']])


In [5]:
X_train.shape

(1996, 2048)

In [7]:
# Split into 80% data and assess performance 

from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

import time
t=time.time()

mlb = MultiLabelBinarizer()
y_ptrain= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

random_state = np.random.RandomState(0)
X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(X_train, y_ptrain, test_size=.2,random_state=random_state)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
classifier.fit(X_ptrain, y_ptrain)

y_ppredict = classifier.predict(X_ptest)

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  186.1 sec


In [8]:
print "Samples of predicted labels (in binary matrix):\n", y_ppredict[0:3]
print "\nSamples of predicted labels:\n", mlb.inverse_transform(y_ppredict[0:3])

Samples of predicted labels (in binary matrix):
[[0 0 0 0 0 1 0 0 1]
 [0 0 1 0 0 1 1 0 1]
 [0 1 1 0 0 1 1 0 1]]

Samples of predicted labels:
[(5, 8), (2, 5, 6, 8), (1, 2, 5, 6, 8)]


In [9]:
stats = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
stats.loc["biz count"] = np.append(np.sum(y_ppredict, axis=0), len(y_ppredict))
pd.options.display.float_format = '{:.0f}%'.format
stats.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_ppredict) 
stats

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,68,192,198,179,83,290,309,77,283,400
biz ratio,17%,48%,50%,45%,21%,72%,77%,19%,71%,100%


In [26]:
sns.distplot(statistics.loc["biz count"])

In [18]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptest, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)

F1 score:  0.716533095874
Individual Class F1 score:  [ 0.45410628  0.70984456  0.77435897  0.5326087   0.57754011  0.78888889
  0.84375     0.52910053  0.81368821]


In [14]:
print stats

          attribuite 0 attribuite 1 attribuite 2 attribuite 3 attribuite 4  \
biz count           68          192          198          179           83   
biz ratio          17%          48%          50%          45%          21%   

          attribuite 5 attribuite 6 attribuite 7 attribuite 8 num_biz  
biz count          290          309           77          283     400  
biz ratio          72%          77%          19%          71%    100%  
